# Pre Processing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import ast
import itertools
import pandas as pd


from functools import reduce
from pyspark.sql import DataFrame
from statsmodels.tsa.holtwinters import ExponentialSmoothing as HWES

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20210530205515-0000
KERNEL_ID = a39baeb0-082d-4109-9f86-32d895a63f26


In [2]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [3]:

import ibmos2spark, os
# @hidden_cell

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_7efe33a944e54231997eacfdaad94b04 = 'https://s3-api.us-geo.objectstorage.softlayer.net'
else:
    endpoint_7efe33a944e54231997eacfdaad94b04 = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'

credentials = {
    'endpoint': endpoint_7efe33a944e54231997eacfdaad94b04,
    'service_id': 'iam-ServiceId-8c64e377-89e3-424a-b706-72ea9ab5fd59',
    'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
    'api_key': 'vNrUHeW9-R1HdOM2ld5PDpR96H0VdqryNzcmyVlNFne_'
}

configuration_name = 'os_7efe33a944e54231997eacfdaad94b04_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_data_1 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('carrinho.csv', 'projetoviagrupo1-donotdelete-pr-q8ymlbqhihqlvv'))
df_data_1.take(5)


[Row(Item_Carrinho='1', idcompra='166000194', IDBANDEIRA='49', flagaprovado='0', idcanalvenda='MOBI', idfreteentregatipo='1', idcompraentrega='82437366', status='CAN', Devolucao='0', Parceiro='gp_pla', Campanha='move_sala_de_estar_shopping', idlojista='15', flagenviadomarketplace='0', data='2019-06-01 12:13:00', Hora_Pedido='12:13:00', Dia_Pedido='2019-06-01', Mes_pedido='2019-06', Flag_Atraso='0', Tempo_atraso=None, dataentrega=None, dataprevisao='2019-06-12 00:00:00', idsku='1000067544', Valor='499.0', Frete='11.64', Valor_Com_Frete='510.64', IDCATEGORIA='693', CATEGORIA='Cozinhas', IDDEPARTAMENTO='93', DEPARTAMENTO='Móveis', Boleto='1', Cartao_Presente='0', Credito='0', Debito='0', Outro_Meio_Pagamento='0', Vale='0', Total_Pagamentos_Insucesso='1', Total_Pagamentos_Sucesso='0'),
 Row(Item_Carrinho='2', idcompra='166000194', IDBANDEIRA='49', flagaprovado='0', idcanalvenda='MOBI', idfreteentregatipo='1', idcompraentrega='82437367', status='CAN', Devolucao='0', Parceiro='gp_pla', Campa

In [146]:

comp_vend = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('compilado_vendas.csv', 'projetoviagrupo1-donotdelete-pr-q8ymlbqhihqlvv'))
#comp_vend.take(5)


In [147]:
comp_vend.limit(10).toPandas()

,ano,mes,dia,idlojista,IDBANDEIRA,idsku,IDCATEGORIA,CATEGORIA,IDDEPARTAMENTO,DEPARTAMENTO,Valor,Frete,Valor_Total,pedidos,itens,ENVIOS,Devolucoes,atrasos,media_atraso,itens_entregues,itens_cancelados,itens_fluxo
0,2020,5,13,11297,7,1500521682,82,Ferro de Passar,73,Eletroportáteis,69.9,10.47,80.37,1,1,1,0,0,None,1,0,0
1,2021,1,10,14738,7,1502406322,326,Smartphones,38,Telefones e Celulares,1378.0,65.42,1443.42,2,2,2,0,0,None,2,0,0
2,2020,6,3,10037,7,1001116117,17,Ar-Condicionado,2809,Ar e Ventilação,13592.0,726.6199999999999,14318.62,4,8,4,0,0,None,0,8,0
3,2020,11,15,31644,7,1501545593,326,Smartphones,38,Telefones e Celulares,949.0,49.84,998.84,1,1,1,0,0,None,0,1,0
4,2019,11,30,12231,49,14399564,1817,Femininos,1733,Relógios,49.0,8.48,57.48,1,1,1,0,0,None,1,0,0
5,2020,3,30,10037,7,410931,79,Grill e Sanduicheiras,73,Eletroportáteis,54.9,0.0,54.9,1,1,1,0,0,None,1,0,0
6,2020,6,11,19139,343,1501149064,14,Refrigeradores,13,Eletrodomésticos,2899.99,159.0,3058.99,1,1,1,0,0,None,1,0,0
7,2019,11,22,15307,7,14660726,99,Quarto Infantil e Bebê,93,Móveis,170.0,49.0,219.0,1,1,1,0,0,None,1,0,0
8,2020,7,15,10037,7,55006488,2,Televisores,1,TV e Vídeo,4398.0,0.0,4398.0,2,2,2,0,1,1.0,2,0,0
9,2020,1,11,20274,49,1500325191,2032,Tablet,2031,Tablets,199.0,6.86,205.86,1,1,1,0,0,None,0,1,0


In [4]:
#df = spark.read.parquet('/mnt/gen2ds/app/analytics/ClienteCarrinho/')
df = df_data_1

In [149]:
comp_vend.createOrReplaceTempView('comp_vend')

In [8]:
pd.options.display.max_columns = 999

In [38]:
spark.sql('select * from comp_vend limit 5').toPandas()

,ano,mes,dia,idlojista,IDBANDEIRA,idsku,IDCATEGORIA,CATEGORIA,IDDEPARTAMENTO,DEPARTAMENTO,Valor,Frete,Valor_Total,pedidos,itens,ENVIOS,atrasos,media_atraso,itens_entregues,itens_cancelados,itens_fluxo
0,2019,9,4,12299,7,7314434,3285,Acessórios e Inovações,3285,Acessórios e Inovações,598.0,99.98,697.98,2,2,2,0,None,2,0,0
1,2021,4,10,10172,7,1507370720,97,Sala de Estar,93,Móveis,527.99,82.4,610.39,1,1,1,0,None,1,0,0
2,2020,4,12,11121,7,11837816,421,Bicicletas,418,Esporte e Lazer,1039.0,80.0,1119.0,1,1,1,0,None,0,1,0
3,2021,4,17,11578,49,13926452,976,Pneus,836,Automotivo,873.8,60.96,934.76,1,2,2,0,None,2,0,0
4,2020,4,15,32711,7,13673627,3359,Decoração,3359,Decoração,159.0,18.02,177.02,1,1,1,0,None,0,1,0


In [82]:
import pyspark.sql.functions as f

In [83]:
comp_vend_agrupLojDia.agg(f.countDistinct('idlojista').alias('QtLojistas')).toPandas()

,QtLojistas
0,7762


In [150]:
comp_vend_agrupBdraDia = spark.sql('''
    select
        ano,
        mes,
        dia,
        IDBANDEIRA, 
        sum(Valor) as valor,
        sum(itens) as itens,
        count(DISTINCT idlojista) as QtLojistas
    from 
        comp_vend
    group by 
        ano,
        mes,
        dia,
        IDBANDEIRA
    order by
        ano,
        mes,
        dia,
        IDBANDEIRA
''')

In [138]:
comp_vend_agrupBdraDia.limit(5).toPandas()

Py4JJavaError: An error occurred while calling o239.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 49.0 failed 4 times, most recent failure: Lost task 1.3 in stage 49.0 (TID 3651, 172.30.155.144, executor 0): java.io.FileNotFoundException: Not found cos://projetoviagrupo1-donotdelete-pr-q8ymlbqhihqlvv.os_7efe33a944e54231997eacfdaad94b04_configs/compilado_vendas.csv
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:124)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:169)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:93)
	at 

In [105]:
#df = df2
comp_vend_agrupBdraDia.printSchema()

root
 |-- ano: string (nullable = true)
 |-- mes: string (nullable = true)
 |-- dia: string (nullable = true)
 |-- IDBANDEIRA: string (nullable = true)
 |-- valor: double (nullable = true)
 |-- itens: double (nullable = true)



In [102]:
pd_comp_vend_agrupBdraDia.count()

ano           2100
mes           2100
dia           2100
IDBANDEIRA    2100
valor         2100
itens         2100
data          2100
tipo          2100
dtype: int64

In [151]:
pd_comp_vend_agrupBdraDia = comp_vend_agrupBdraDia.toPandas()

In [87]:
pd_comp_vend_agrupBdraDia.head(3)

,ano,mes,dia,IDBANDEIRA,valor,itens
0,2019,10,1,343,142844.48,257.0
1,2019,10,1,49,226109.65,412.0
2,2019,10,1,7,478331.17,799.0


In [153]:
pd_comp_vend_agrupBdraDia.count()

ano           2100
mes           2100
dia           2100
IDBANDEIRA    2100
valor         2100
itens         2100
QtLojistas    2100
dtype: int64

In [89]:
pd_comp_vend_agrupBdraDia.dtypes

ano            object
mes            object
dia            object
IDBANDEIRA     object
valor         float64
itens         float64
dtype: object

In [154]:
pd_comp_vend_agrupBdraDia['data'] = pd.to_datetime(
    pd_comp_vend_agrupBdraDia['ano'].astype(str)+'-'+\
    pd_comp_vend_agrupBdraDia['mes'].astype(str)+'-'+\
    pd_comp_vend_agrupBdraDia['dia'].astype(str)
)

In [155]:
pd_comp_vend_agrupBdraDia.dtypes

ano                   object
mes                   object
dia                   object
IDBANDEIRA            object
valor                float64
itens                float64
QtLojistas             int64
data          datetime64[ns]
dtype: object

In [156]:
pd_comp_vend_agrupBdraDia["tipo"] = "realizado"

In [157]:
pd_comp_vend_agrupBdraDia['valor'] = pd.to_numeric(pd_comp_vend_agrupBdraDia['valor'],downcast='signed')

In [158]:
pd_comp_vend_agrupBdraDia.head()

,ano,mes,dia,IDBANDEIRA,valor,itens,QtLojistas,data,tipo
0,2019,10,1,343,142844.48,257.0,112,2019-10-01,realizado
1,2019,10,1,49,226109.65,412.0,140,2019-10-01,realizado
2,2019,10,1,7,478331.17,799.0,216,2019-10-01,realizado
3,2019,10,10,343,149396.04,248.0,92,2019-10-10,realizado
4,2019,10,10,49,153197.11,294.0,132,2019-10-10,realizado


In [113]:
pd_comp_vend_agrupLojDia.IDBANDEIRA.unique()

array(['343', '49', '7'], dtype=object)

In [120]:
pd_comp_vend_agrupBdraDia.nunique()

ano              3
mes             12
dia             31
IDBANDEIRA       3
valor         2100
itens         1031
data           700
tipo             1
dtype: int64

In [121]:
pd_comp_vend_agrupBdraDia.IDBANDEIRA.describe()

count     2100
unique       3
top         49
freq       700
Name: IDBANDEIRA, dtype: object

In [124]:
pd_comp_vend_agrupBdraDia.loc[pd_comp_vend_agrupBdraDia.IDBANDEIRA.isnull()]

,ano,mes,dia,IDBANDEIRA,valor,itens,data,tipo


In [125]:
pd_comp_vend_agrupBdraDia.IDBANDEIRA.unique()

array(['343', '49', '7'], dtype=object)

In [159]:
# Agora precisamos criar 9 DFs, com as combinações de bandeira e device

# CB
#df_cb_app = df[(df.bandeira == 'CASASBAHIA') ]
#df_cb_msite = df[(df.bandeira == 'PONTOFRIO') ]
#df_cb_desktop = df[(df.bandeira == 'EXTRA') ]

pd_comp_vend_agrupBdraDia_343 = pd_comp_vend_agrupBdraDia.loc[(pd_comp_vend_agrupBdraDia.IDBANDEIRA == '343') ]
pd_comp_vend_agrupBdraDia_49 = pd_comp_vend_agrupBdraDia.loc[(pd_comp_vend_agrupBdraDia.IDBANDEIRA == '49') ]
pd_comp_vend_agrupBdraDia_7 = pd_comp_vend_agrupBdraDia.loc[(pd_comp_vend_agrupBdraDia.IDBANDEIRA == '7') ]

In [127]:
pd_comp_vend_agrupBdraDia.nunique()

ano              3
mes             12
dia             31
IDBANDEIRA       3
valor         2100
itens         1031
data           700
tipo             1
dtype: int64

In [161]:
pd_comp_vend_agrupBdraDia_343.head()

,ano,mes,dia,IDBANDEIRA,valor,itens,QtLojistas,data,tipo
0,2019,10,1,343,142844.48,257.0,112,2019-10-01,realizado
3,2019,10,10,343,149396.04,248.0,92,2019-10-10,realizado
6,2019,10,11,343,113787.60,237.0,103,2019-10-11,realizado
9,2019,10,12,343,122784.02,206.0,89,2019-10-12,realizado
12,2019,10,13,343,132259.77,245.0,96,2019-10-13,realizado


In [128]:
pd_comp_vend_agrupBdraDia_49.nunique()

ano             3
mes            12
dia            31
IDBANDEIRA      1
valor         700
itens         351
data          700
tipo            1
dtype: int64

In [129]:
pd_comp_vend_agrupBdraDia_343.count()

ano           700
mes           700
dia           700
IDBANDEIRA    700
valor         700
itens         700
data          700
tipo          700
dtype: int64

In [131]:
def split_df_per_metrics(df):

    df_receita = df[['data', 'valor']].set_index('data')
    df_receita['valor'] = df_receita.astype('float64')

    return  df_receita

In [178]:
receita_343 = split_df_per_metrics(pd_comp_vend_agrupBdraDia_343)
receita_49 = split_df_per_metrics(pd_comp_vend_agrupBdraDia_49)
receita_7 = split_df_per_metrics(pd_comp_vend_agrupBdraDia_7)

In [179]:
receita_343.head()

,valor
data,
2019-10-01,142844.48
2019-10-10,149396.04
2019-10-11,113787.60
2019-10-12,122784.02
2019-10-13,132259.77


In [142]:
receita_343.head()

,valor
data,
2019-10-01,142844.48
2019-10-10,149396.04
2019-10-11,113787.60
2019-10-12,122784.02
2019-10-13,132259.77


In [143]:
receita_49.head()

,valor
data,
2019-10-01,226109.65
2019-10-10,153197.11
2019-10-11,150320.16
2019-10-12,113560.75
2019-10-13,162128.75


In [63]:
def train_test_split(df, time_period):
    df_train = df.iloc[:-time_period]
    df_test = df.iloc[-time_period:]
    return df_train, df_test

In [68]:
df_343.Dia_Pedido.max()

'2021-04-30'

In [64]:
def best_model(df_train, df_test):
    # Cria o dict vazio que vai ser populado com os hyperparams
    dict_metrics = {}

    # Popula o dict com os hyperparams e o mape
    for i in permutations_dicts:
        model = HWES(df_train, seasonal_periods=i['seasonal_periods'], seasonal=i['seasonal'], trend=i['trend'], freq='D')
        fitted = model.fit()
        test_forecast = fitted.forecast(len(df_test))
        dict_metrics[str(i)] = str(mean_absolute_percentage_error(test_forecast, df_test))

    # Retorna o dict hyperparams que tem o menor mape    
    best_hyperparams = min(dict_metrics, key=dict_metrics.get)
    best_hyperparams = ast.literal_eval(best_hyperparams)

    # Otimiza o modelo usando o dict hyperparam do menor mape
    best_model = HWES(df_train, seasonal_periods=best_hyperparams.get('seasonal_periods'), seasonal=best_hyperparams.get('seasonal'), trend=best_hyperparams.get('trend'), freq='D')

    # Retorna o modelo fittado
    best_model_fitted = best_model.fit()

    return best_model_fitted

In [78]:
receita_comp_vend_agrupLojDia.head()

,valor
data,
2019-10-01,2707.20
2019-10-01,12393.00
2019-10-01,716.92
2019-10-01,2548.00
2019-10-01,105.93


In [ ]:
rece

In [255]:
receita = {
    '343':receita_343,
    '49':receita_49,
    '7':receita_7
}

In [183]:
receita['343'].head()

,valor
data,
2019-10-01,142844.48
2019-10-10,149396.04
2019-10-11,113787.60
2019-10-12,122784.02
2019-10-13,132259.77


In [256]:
# Receita
# CB
# best_model_cb_app_receita = HWES(receita_cb_app, seasonal_periods=14, seasonal='add', trend='add', freq='D').fit()
# best_model_cb_msite_receita = HWES(receita_cb_msite, seasonal_periods=14, seasonal='add', trend='add', freq='D').fit()
# best_model_cb_desktop_receita = HWES(receita_cb_desktop, seasonal_periods=14, seasonal='add', trend='add', freq='D').fit()

best_model_receita = {}
for bdra in receita:
    best_model_receita[bdra] = HWES(
        receita[bdra],
        seasonal_periods=30,
        seasonal='add',
        trend='add'
    ).fit()

/opt/ibm/conda/miniconda/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/opt/ibm/conda/miniconda/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:222: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  ' forecasting.', ValueWarning)
/opt/ibm/conda/miniconda/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/opt/ibm/conda/miniconda/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:222: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.

In [257]:
from datetime import datetime
from datetime import date

def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days)

In [258]:
# Forecasting Receita
# forecast_cb_app_receita = best_model_cb_app_receita.forecast(FUTURE_DAYS)
# forecast_cb_msite_receita = best_model_cb_msite_receita.forecast(FUTURE_DAYS)
# forecast_cb_desktop_receita = best_model_cb_desktop_receita.forecast(FUTURE_DAYS)
forecast = {}
for bdra in receita:
    forecast[bdra] = best_model_receita[bdra].forecast(7)

/opt/ibm/conda/miniconda/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
/opt/ibm/conda/miniconda/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
/opt/ibm/conda/miniconda/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


In [259]:
def return_to_df(series, metric, bandeira):
    df_out = series.to_frame().reset_index()
    df_out.columns=['dia_pedido', metric]
    df_out['bandeira'] = bandeira
    return df_out

In [260]:
forecast['49'] = return_to_df(forecast['49'], 'valor', '49')
forecast['343'] = return_to_df(forecast['343'], 'valor', '343')
forecast['7'] = return_to_df(forecast['7'], 'valor', '7')

In [261]:
forecast['49']

,dia_pedido,valor,bandeira
0,700,431539.276313,49
1,701,441050.165328,49
2,702,449808.604216,49
3,703,429526.798028,49
4,704,454965.115579,49
5,705,427332.372662,49
6,706,447879.229696,49


In [262]:
dias

[datetime.date(2021, 5, 1),
 datetime.date(2021, 5, 2),
 datetime.date(2021, 5, 3),
 datetime.date(2021, 5, 4),
 datetime.date(2021, 5, 5),
 datetime.date(2021, 5, 6),
 datetime.date(2021, 5, 7)]

In [232]:
forecast['343']

,dia_pedido,valor,bandeira
0,2021-05-01,1.727825e+06,343
1,2021-05-02,1.749532e+06,343
2,2021-05-03,1.738578e+06,343
3,2021-05-04,1.775349e+06,343
4,2021-05-05,1.782812e+06,343
5,2021-05-06,1.780677e+06,343
6,2021-05-07,1.795558e+06,343


In [263]:
for bdra in forecast:
    forecast[bdra]['dia_pedido'] = dias

In [264]:
df_receita_forecast = pd.concat([
    forecast['343'],
    forecast['49'],
    forecast['7']
  ], join="inner")

df_receita_forecast.head()

,dia_pedido,valor,bandeira
0,2021-05-01,1.727825e+06,343
1,2021-05-02,1.749532e+06,343
2,2021-05-03,1.738578e+06,343
3,2021-05-04,1.775349e+06,343
4,2021-05-05,1.782812e+06,343


In [265]:
df_receita_forecast["tipo"] = 'forecasting'

In [266]:
pd_comp_vend_agrupBdraDia.head()

,ano,mes,dia,IDBANDEIRA,valor,itens,QtLojistas,data,tipo
0,2019,10,1,343,142844.48,257.0,112,2019-10-01,realizado
1,2019,10,1,49,226109.65,412.0,140,2019-10-01,realizado
2,2019,10,1,7,478331.17,799.0,216,2019-10-01,realizado
3,2019,10,10,343,149396.04,248.0,92,2019-10-10,realizado
4,2019,10,10,49,153197.11,294.0,132,2019-10-10,realizado


In [267]:
df_receita_forecast.head()

,dia_pedido,valor,bandeira,tipo
0,2021-05-01,1.727825e+06,343,forecasting
1,2021-05-02,1.749532e+06,343,forecasting
2,2021-05-03,1.738578e+06,343,forecasting
3,2021-05-04,1.775349e+06,343,forecasting
4,2021-05-05,1.782812e+06,343,forecasting


In [268]:
df_receita_forecast['dia_pedido'] = df_receita_forecast['dia_pedido'].astype(str)

In [269]:
df_receita_forecast['ano'] = df_receita_forecast['dia_pedido'].str[:4].astype(int)
df_receita_forecast['mes'] = df_receita_forecast['dia_pedido'].str[5:7].astype(int)
df_receita_forecast['dia'] = df_receita_forecast['dia_pedido'].str[8:].astype(int)

In [270]:
df_receita_forecast.head()

,dia_pedido,valor,bandeira,tipo,ano,mes,dia
0,2021-05-01,1.727825e+06,343,forecasting,2021,5,1
1,2021-05-02,1.749532e+06,343,forecasting,2021,5,2
2,2021-05-03,1.738578e+06,343,forecasting,2021,5,3
3,2021-05-04,1.775349e+06,343,forecasting,2021,5,4
4,2021-05-05,1.782812e+06,343,forecasting,2021,5,5


In [296]:
pd_comp_vend_agrupBdraDia['data'] = pd_comp_vend_agrupBdraDia['data'].astype(str)

In [297]:
pd_comp_vend_agrupBdraDia.head()

,ano,mes,dia,IDBANDEIRA,valor,itens,QtLojistas,data,tipo
0,2019,10,1,343,142844.48,257.0,112,2019-10-01,realizado
1,2019,10,1,49,226109.65,412.0,140,2019-10-01,realizado
2,2019,10,1,7,478331.17,799.0,216,2019-10-01,realizado
3,2019,10,10,343,149396.04,248.0,92,2019-10-10,realizado
4,2019,10,10,49,153197.11,294.0,132,2019-10-10,realizado


In [298]:
df_forecast = df_forecast[[
    'ano',
    'mes',
    'dia',
    'bandeira',
    'valor',
    'dia_pedido',
    'tipo'
]].rename(columns={'bandeira':'IDBANDEIRA','dia_pedido':'data'})

In [299]:
df_forecast.head()

,ano,mes,dia,IDBANDEIRA,valor,data,tipo
0,2021,5,1,343,1.727825e+06,2021-05-01,forecasting
1,2021,5,2,343,1.749532e+06,2021-05-02,forecasting
2,2021,5,3,343,1.738578e+06,2021-05-03,forecasting
3,2021,5,4,343,1.775349e+06,2021-05-04,forecasting
4,2021,5,5,343,1.782812e+06,2021-05-05,forecasting


In [300]:
pd_comp_vend_agrupBdraDia.head()

,ano,mes,dia,IDBANDEIRA,valor,itens,QtLojistas,data,tipo
0,2019,10,1,343,142844.48,257.0,112,2019-10-01,realizado
1,2019,10,1,49,226109.65,412.0,140,2019-10-01,realizado
2,2019,10,1,7,478331.17,799.0,216,2019-10-01,realizado
3,2019,10,10,343,149396.04,248.0,92,2019-10-10,realizado
4,2019,10,10,49,153197.11,294.0,132,2019-10-10,realizado


In [313]:
qtloj = {}
for bdra in forecast:
    qtloj[bdra] = pd_comp_vend_agrupBdraDia.loc[
        (pd_comp_vend_agrupBdraDia.data == '2021-04-30')&\
        (pd_comp_vend_agrupBdraDia.IDBANDEIRA == bdra),
        'QtLojistas'
    ].item()

In [317]:
df_final = pd.concat([pd_comp_vend_agrupBdraDia, df_forecast], join='inner')

In [321]:
df_final = df_final.merge(
    pd_comp_vend_agrupBdraDia[['data','tipo','itens','QtLojistas']],
    on=['data','tipo'],
    how='left'
)

In [322]:
df_final

,ano,mes,dia,IDBANDEIRA,valor,data,tipo,itens,QtLojistas
0,2019,10,1,343,1.428445e+05,2019-10-01,realizado,257.0,112.0
1,2019,10,1,343,1.428445e+05,2019-10-01,realizado,412.0,140.0
2,2019,10,1,343,1.428445e+05,2019-10-01,realizado,799.0,216.0
3,2019,10,1,49,2.261096e+05,2019-10-01,realizado,257.0,112.0
4,2019,10,1,49,2.261096e+05,2019-10-01,realizado,412.0,140.0
...,...,...,...,...,...,...,...,...,...
6316,2021,5,3,7,1.915766e+06,2021-05-03,forecasting,NaN,NaN
6317,2021,5,4,7,2.048467e+06,2021-05-04,forecasting,NaN,NaN
6318,2021,5,5,7,2.062251e+06,2021-05-05,forecasting,NaN,NaN
6319,2021,5,6,7,1.979660e+06,2021-05-06,forecasting,NaN,NaN


In [324]:
df_final['valorMedioPorLojista'] = df_final['valor']/df_final['QtLojistas']

In [325]:
df_final.head()

,ano,mes,dia,IDBANDEIRA,valor,data,tipo,itens,QtLojistas,valorMedioPorLojista
0,2019,10,1,343,142844.48,2019-10-01,realizado,257.0,112.0,1275.397143
1,2019,10,1,343,142844.48,2019-10-01,realizado,412.0,140.0,1020.317714
2,2019,10,1,343,142844.48,2019-10-01,realizado,799.0,216.0,661.317037
3,2019,10,1,49,226109.65,2019-10-01,realizado,257.0,112.0,2018.836161
4,2019,10,1,49,226109.65,2019-10-01,realizado,412.0,140.0,1615.068929


In [327]:
for bdra in qtloj:
    df_final.loc[
        (df_final.tipo == 'forecasting')&\
        (df_final.IDBANDEIRA == bdra),
        'valorMedioPorLojista'
    ] = df_final.loc[
        (df_final.tipo == 'forecasting')&\
        (df_final.IDBANDEIRA == bdra),
        'valor'
    ]/qtloj[bdra]

In [328]:
df_final

,ano,mes,dia,IDBANDEIRA,valor,data,tipo,itens,QtLojistas,valorMedioPorLojista
0,2019,10,1,343,1.428445e+05,2019-10-01,realizado,257.0,112.0,1275.397143
1,2019,10,1,343,1.428445e+05,2019-10-01,realizado,412.0,140.0,1020.317714
2,2019,10,1,343,1.428445e+05,2019-10-01,realizado,799.0,216.0,661.317037
3,2019,10,1,49,2.261096e+05,2019-10-01,realizado,257.0,112.0,2018.836161
4,2019,10,1,49,2.261096e+05,2019-10-01,realizado,412.0,140.0,1615.068929
...,...,...,...,...,...,...,...,...,...,...
6316,2021,5,3,7,1.915766e+06,2021-05-03,forecasting,NaN,NaN,5552.944000
6317,2021,5,4,7,2.048467e+06,2021-05-04,forecasting,NaN,NaN,5937.586219
6318,2021,5,5,7,2.062251e+06,2021-05-05,forecasting,NaN,NaN,5977.539686
6319,2021,5,6,7,1.979660e+06,2021-05-06,forecasting,NaN,NaN,5738.144710


In [ ]:
def upload_file_cos(cos, local_file_name, credentials,key):  
    
    try:
        res=cos.upload_file(Filename=local_file_name+key, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print(key+' - File Uploaded -'+ (datetime.now() - timedelta(hours=3)).strftime("%d/%m/%Y %H:%M:%S"))

In [329]:
credentials_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-8c64e377-89e3-424a-b706-72ea9ab5fd59',
    'IBM_API_KEY_ID': 'vNrUHeW9-R1HdOM2ld5PDpR96H0VdqryNzcmyVlNFne_',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'projetoviagrupo1-donotdelete-pr-q8ymlbqhihqlvv',
    'CES' : 'compraentregastatus.csv',
    'SL' : 'skulojista.csv',
    'CE' : 'compraentrega.csv',
    'C' : 'compra.csv',
    'CESKU' : 'compraentregasku.csv',
    'CFPG' : 'compraformapagamento.csv',
    'LOJ' : 'lojista.csv',
    'CAT': 'skucategoria.csv',
    'NPS' : 'visaolojistavendanps.csv',
    'FPG' : 'formapagamento.csv'
}

In [332]:
import ibm_boto3
from botocore.client import Config
cgsClient = ibm_boto3.client(service_name='s3',
    ibm_api_key_id = credentials_1['IBM_API_KEY_ID'],
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

In [333]:
path = '/home/spark/shared/tmp/'

In [ ]:
df_final.toPandas().to_json('/home/spark/shared/tmp/estim_valor.json',orient="records")
upload_file_cos(cgsClient, path,credentials_1,'estim_valor.json')